<a href="https://colab.research.google.com/github/Davidospina1000/Data-Integration/blob/main/Reto_4_Integraci%C3%B3n_Din%C3%A1mica_de_los_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Integración Dinámica de los Datos**

0. Se cargan las librerías

In [ ]:
import numpy as np
import pandas as pd

1. Se cargan los datos de trabajo

In [ ]:
# Cargar la base de datos principal
nxl = '/content/4. DB_Fintechs (USD).xlsx'
XDB = pd.read_excel(nxl, sheet_name=0)  # Base de datos observada
XDB = XDB.iloc[:, [1, 10, 11, 25]].dropna()   # Ajustar a 4 variables
XDB = np.array(XDB)

2. Se procede con el proceso de clusterización Estática (K-Means)

In [ ]:
# 1. Crear semillas iniciales
XDBM = np.max(XDB, axis=0)
XDBm = np.min(XDB, axis=0)
XC = np.zeros((5, 4))  # Ajustar a 4 variables

for i in range(5):
    for j in range(4):
        XC[i, j] = np.random.uniform(XDBm[j], XDBM[j])

print("Semillas iniciales:", XC.astype(int))

XCd = np.zeros((5, 4))  # Clusters después de la clasificación
ND = np.zeros((5, 1))
clusters_XDB = np.zeros((len(XDB), 1))

# 2. Clasificación de los datos en clusters iniciales
for k in range(len(XDB)):
    d = np.sum(np.abs(XC[:, :] - XDB[k, :]), axis=1)
    fila = np.argmin(d)
    XCd[fila, :] += XDB[k, :]
    ND[fila, 0] += 1
    clusters_XDB[k, 0] = fila

#fila=np.where(ND==0)[0][0]
#ND[fila, 0] = 1

print("El número de datos por cluster es:\n", ND)

XCd = XCd / ND  # Promediar valores de los clusters

# Correlación entre variables
dfcor = pd.DataFrame(XDB).corr()
print("La correlación entre las variables es:\n", dfcor)


Semillas iniciales: [[  53 1034  841 2203]
 [  61  282  145  114]
 [  65   87  559  595]
 [  53  713 1227  140]
 [  40 2425 1628 2482]]
El número de datos por cluster es:
 [[146.]
 [565.]
 [347.]
 [380.]
 [ 62.]]
La correlación entre las variables es:
           0         1         2         3
0  1.000000 -0.033098 -0.037194 -0.039985
1 -0.033098  1.000000  0.888146  0.818815
2 -0.037194  0.888146  1.000000  0.522718
3 -0.039985  0.818815  0.522718  1.000000


3. Integración dinámica (K-medoids)

In [ ]:
# Función para integración dinámica de datos externos
def integrar_datos(index, XCd):
    XDBe = pd.read_excel(nxl, sheet_name=index)
    XDBe = np.array(XDBe.iloc[:, [1, 10, 11, 25]].dropna())  # Ajustar a 4 variables

    XCe = np.copy(XCd)
    NDe = np.zeros((5, 1))
    nf = np.zeros((len(XDBe), 1))

    for k in range(len(XDBe)):
        de = np.sqrt(np.sum((XCe[:, :] - XDBe[k, :])**2, axis=1))
        fila = np.argmin(de)
        nf[k, 0] = fila
        XCe[fila, :] = (XCe[fila, :] + XDBe[k, :]) / 2
        NDe[fila, 0] += 1

    dfinf = pd.DataFrame(np.column_stack((nf, XDBe)), columns=['Cluster', 'Edad', 'Ingresos', 'Egresos', 'Monto'])

    return NDe, XCe, dfinf

In [ ]:
int1 = integrar_datos(1, XCd); int2 = integrar_datos(2, int1[1]); int3 = integrar_datos(3, int2[1])

In [ ]:
print("El número de datos externos integrados por cluster es:\n", int1[0])
print("La estructura de los clusters antes:\n", pd.DataFrame(XCd))
print("La estructura de los clusters después:\n", pd.DataFrame(int1[1]))
print("Los cambios porcentuales fueron:\n", (XCd - int1[1]) / XCd)

El número de datos externos integrados por cluster es:
 [[ 64.]
 [532.]
 [527.]
 [236.]
 [142.]]
La estructura de los clusters antes:
            0            1            2            3
0  41.527397  1308.613067   823.322087  1330.142650
1  41.879646   263.531259   229.122694   187.589760
2  40.340058   719.369159   521.000045   634.879418
3  40.784211  1036.008067  1111.439965   554.383382
4  38.967742  2071.562032  1937.517190  1459.102881
La estructura de los clusters después:
            0            1            2            3
0  39.404900  2045.730927  2321.892211  1065.205838
1  46.407853  1212.232675   721.627206  1578.257527
2  42.747108   231.627334   235.209577   134.707855
3  31.718732  1079.250088   907.843953   726.503718
4  48.812462  1817.979170   928.699984  2669.071993
Los cambios porcentuales fueron:
 [[ 0.05111077 -0.56328175 -1.82015052  0.19917925]
 [-0.1081243  -3.59995781 -2.14952305 -7.41334583]
 [-0.05966899  0.67801325  0.54854212  0.78782136]
 [ 0.22227912 

In [ ]:
print("El número de datos externos integrados por cluster es:\n", int2[0])
print("La estructura de los clusters antes:\n", pd.DataFrame(XCd))
print("La estructura de los clusters después:\n", pd.DataFrame(int2[1]))
print("Los cambios porcentuales fueron:\n", (XCd - int2[1]) / XCd)

El número de datos externos integrados por cluster es:
 [[146.]
 [286.]
 [500.]
 [173.]
 [395.]]
La estructura de los clusters antes:
            0            1            2            3
0  41.527397  1308.613067   823.322087  1330.142650
1  41.879646   263.531259   229.122694   187.589760
2  40.340058   719.369159   521.000045   634.879418
3  40.784211  1036.008067  1111.439965   554.383382
4  38.967742  2071.562032  1937.517190  1459.102881
La estructura de los clusters después:
            0            1            2            3
0  28.497881  1357.987255  1261.832265   861.020109
1  43.534835  1604.103237   868.245778  2353.786509
2  33.074653   740.355358   806.333898   465.304359
3  36.155829  1891.145045  1992.731772  1085.650252
4  43.069963   201.151828   155.122095   195.292452
Los cambios porcentuales fueron:
 [[  0.31375711  -0.03773017  -0.53261073   0.35268589]
 [ -0.03952251  -5.08695622  -2.78943597 -11.54752128]
 [  0.18010398  -0.02917306  -0.5476657    0.26709806]
 [

In [ ]:
print("El número de datos externos integrados por cluster es:\n", int3[0])
print("La estructura de los clusters antes:\n", pd.DataFrame(XCd))
print("La estructura de los clusters después:\n", pd.DataFrame(int3[1]))
print("Los cambios porcentuales fueron:\n", (XCd - int3[1]) / XCd)

El número de datos externos integrados por cluster es:
 [[101.]
 [252.]
 [181.]
 [284.]
 [524.]]
La estructura de los clusters antes:
            0            1            2            3
0  41.527397  1308.613067   823.322087  1330.142650
1  41.879646   263.531259   229.122694   187.589760
2  40.340058   719.369159   521.000045   634.879418
3  40.784211  1036.008067  1111.439965   554.383382
4  38.967742  2071.562032  1937.517190  1459.102881
La estructura de los clusters después:
            0            1            2            3
0  54.543037  1915.567941  1610.841440  1696.553566
1  38.252837  1099.184683   568.000873  1307.052336
2  30.869978  1429.357704  1284.901542   778.417710
3  48.715781   892.656049   624.376553   590.965743
4  40.759753   255.944668   194.571523   240.318859
Los cambios porcentuales fueron:
 [[-0.31342297 -0.46381539 -0.95651431 -0.27546738]
 [ 0.08660075 -3.17098406 -1.47902494 -5.96761024]
 [ 0.23475623 -0.98695995 -1.46622156 -0.22608749]
 [-0.19447649 

In [ ]:
df_XDB = pd.DataFrame(XDB, columns=['Edad', 'Ingresos', 'Egresos', 'Monto'])
df_XDB.insert(0, 'Cluster', clusters_XDB.astype(int))

dfint_total = pd.concat([df_XDB, int1[2], int2[2], int3[2]])

dfint_total

,Cluster,Edad,Ingresos,Egresos,Monto
0,1.0,33.0,496.87275,550.117556,250.209001
1,2.0,61.0,518.90575,240.179333,574.469987
2,0.0,29.0,1305.97300,769.749778,1469.739871
3,1.0,39.0,194.06275,188.418222,76.842043
4,3.0,38.0,883.16350,1059.917333,440.780558
...,...,...,...,...,...
1337,3.0,61.0,944.02575,470.807333,644.978049
1338,1.0,39.0,994.38275,508.646667,1167.670717
1339,4.0,37.0,429.39000,285.619111,256.489696
1340,4.0,29.0,360.14525,198.219111,444.339274


In [ ]:
dfint_total.to_excel('dfint_total.xlsx', index=False)

# Análisis de resultados

En la integración de la base de datos externa 1, el clúster 1 incorporó la mayor cantidad de datos externos (532), pero también presentó las mayores variaciones en ingresos (359%), egresos (215%) y monto (741%). Le siguió el clúster 2 con 527 datos integrados, aunque con variaciones menores, pero aún significativas en ingresos (67,8%), egresos (54,8%) y monto (78,8%). Por su parte, el clúster 0, con solo 64 datos, mostró una variación del 182% en egresos.

De manera similar, en la integración incremental de la Fintech 2, el clúster 2 fue el de mayor integración (500 datos), con variaciones moderadas, salvo en egresos, donde la estructura aumentó un 54,7%. En segundo lugar, el clúster 4 integró una cantidad considerable de datos y presentó variaciones en ingresos (90%), egresos (91,9%) y monto (86,6%). El clúster 3, con 173 datos, mostró cambios en las mismas variables tras la integración (-) con incrementos del 82,5%, 79,3% y 95,8%, respectivamente. Finalmente, el clúster 1, con 286 datos, tuvo los mayores aumentos: ingresos 500%, egresos 279% y monto 1154%.

Para la Fintech 3, el clúster 4 lideró la integración con 524 datos, con variaciones en ingresos (87%), egresos (89,9%) y monto (83,5%), mientras que la edad apenas aumentó un 4,6%. El clúster 1, con 252 datos, registró las mayores variaciones en ingresos (317%), egresos (147%) y monto (596%). De manera similar, el clúster 2 (181 datos) presentó cambios en ingresos (-98,6%) y egresos (-146,6%), reflejando un aumento en la estructura. Finalmente, el clúster 0 mostró variaciones, destacando la de egresos (-95,6%).

Estos resultados sugieren implementar medidas para controlar la integración de datos externos, eliminando los datos de los clústeres con mayores variaciones y priorizando aquellos que mejor se alinean con la estructura de los datos internos.


